In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
import pandas as pd

from moge.network.heterogeneous_network import HeterogeneousNetwork

from sklearn.model_selection import ParameterGrid, ParameterSampler

In [2]:
import pickle

# READ
with open('moge/data/LMN_future_recall/TRAIN/Interactions_Affinity/LMN_mirtarbase_biogrid_starbase_lncrna2target_lncrinter.train.pickle', 'rb') as file:
    network = pickle.load(file)
    
with open('moge/data/LMN_future_recall/TEST/Interactions_Affinity/LMN_mirtarbase_biogrid_starbase_lncrna2target_lncrinter.test.pickle', 'rb') as file:
    network_val = pickle.load(file)

# Set parameter space

In [3]:
from scipy.stats.distributions import uniform
parameters = {
    "d": [128, 256, 512],
    "lr": [0.001, 0.0005],
    # "margin": [0.1, 0.2, 0.5],
    "compression_func": ["sqrt", "log", "sqrt3"],
    "negative_sampling_ratio": [2.0, 5.0, 15.0],
#     "directed_proba": [0.5, 0.8],
    "max_length": [1400, 1800, 2000],
    "truncating": ["random", "post"],
    
    "conv1_kernel_size": [6, 12, 18],
    "max1_pool_size": [3, 4, 6],
    "conv2_kernel_size": [None, 2, 4, 6],
    "max2_pool_size": [2, 4, 6],
    "lstm_unit_size": [160, 320],
    "dense1_unit_size": [256, 512, 1024],
    "dense2_unit_size": [None, 256, 512],
}

# Train Model

In [4]:
from moge.embedding.siamese_graph_embedding import SiameseGraphEmbedding
from moge.embedding.siamese_triplet_online_embedding import SiameseOnlineTripletGraphEmbedding


siamese = SiameseOnlineTripletGraphEmbedding(batch_size=386, epochs=1, verbose=False)

# siamese = SiameseTripletGraphEmbedding(batch_size=256, epochs=1, verbose=False)


Using TensorFlow backend.


In [ ]:
best_score = float("inf")
X_params = []
y = []

for g in ParameterSampler(parameters, n_iter=300):
    print(len(X_params), ":", g)
    siamese.set_params(**g)
    
    try:
        siamese.learn_embedding(network, network_val=network_val, multi_gpu=False, 
                            subsample=False, n_steps=500, validation_steps=None, 
                            validation_make_data=False, rebuild_model=True,
                            seed=42)
    except Exception as e:
        print("Failed!!!")
        continue
    except KeyboardInterrupt as e:
        continue
    
    current_score = siamese.hist.history['val_loss'][-1]
    X_params.append(g)
    y.append(current_score)
    
    if current_score < best_score:
        best_score = current_score
        best_grid = g
        best_history = siamese.hist.history
        print("Score:", best_score, "\n")
    


0 : {'truncating': 'post', 'negative_sampling_ratio': 5.0, 'max_length': 1400, 'max2_pool_size': 2, 'max1_pool_size': 3, 'lstm_unit_size': 320, 'lr': 0.001, 'dense2_unit_size': 256, 'dense1_unit_size': 1024, 'd': 128, 'conv2_kernel_size': 6, 'conv1_kernel_size': 18, 'compression_func': 'sqrt'}
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
directed_loss Tensor("online_triplet_loss_1/Mean_2:0", shape=(), dtype=float32, device=/device:GPU:0)
undirected_loss Tensor("online_triplet_loss_1/mul_10:0", shape=(), dtype=float32, device=/device:GPU:0)
Epoch 1/1
500/500 [==============================] - 867s 2s/step - loss: 0.0483 - val_loss: 0.0298
Score: 0.02982184652239084 

1 : {'truncating': 'random', 'negative_sampling_ratio': 15.0, 'max_length': 1400, 'max2_pool_size': 6, 'max1_pool_size': 4, 'lstm_unit_size': 320, 'lr': 0.0005, 'dense2_unit_size': None, 'dense1_unit_size': 512, 'd': 512, 'conv2_kernel_size': 2, 'conv1_kernel_size': 18, 

Process ForkPoolWorker-59:
Process ForkPoolWorker-56:
Process ForkPoolWorker-54:
Process ForkPoolWorker-51:
Process ForkPoolWorker-58:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/a

Stop training
3 : {'truncating': 'random', 'negative_sampling_ratio': 15.0, 'max_length': 1400, 'max2_pool_size': 4, 'max1_pool_size': 6, 'lstm_unit_size': 320, 'lr': 0.0005, 'dense2_unit_size': 512, 'dense1_unit_size': 256, 'd': 128, 'conv2_kernel_size': None, 'conv1_kernel_size': 12, 'compression_func': 'log'}


In [ ]:
print(best_score)

In [ ]:
print(best_grid)

In [ ]:
print(best_history)